In [1]:
# savePath = "/content/gdrive/MyDrive/Colab Notebooks/fintech_edu_2025/중간고사/models/gemma_tune_2025_05_05_10"

In [2]:
try:
    import google.colab
    inColab = True
except ImportError:
    inColab = False

In [ ]:
if inColab == True:
    !pip install -U pandas==2.2.2 numpy==2.0.2 scipy==1.14.1 accelerate==1.6.0 peft==0.15.2 bitsandbytes==0.45.5 transformers==4.51.3 trl==0.16.1 datasets==3.5.0 tensorboard==2.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from transformers import AutoConfig,AutoModel
import torch
from peft import PeftModel, PeftConfig

In [ ]:
if inColab == True:
    from google.colab import drive
    drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import huggingface_hub
huggingface_hub.login()

# 1. Load base and new model

In [ ]:
# from transformers import AutoConfig, AutoModel, AutoTokenizer
# config = AutoConfig.from_pretrained("your model name", revision=revision)
# model = AutoModel.from_pretrained("your model name", revision=revision)
# tokenizer = AutoTokenizer.from_pretrained("your model name", revision=revision)

### ★★★ 수정 포인트 ★★★

In [ ]:
## base 모델
base_model = "google/gemma-7b-it"
## 파인튜닝한 모델 저장 위치 설정
new_model = "/content/gdrive/MyDrive/Colab Notebooks/fintech_edu_2025/중간고사/models/gemma_tune_2025_05_05_10"

In [ ]:
### 베이스모델 불러오기
baseModel = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map= "auto"
    # device_map= {"": 0}
)

### 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
# PEFT 설정 로드
peft_config = PeftConfig.from_pretrained(new_model)
print(peft_config)

# 베이스모델에 어댑터 적용
model = PeftModel.from_pretrained(baseModel, new_model)

LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='google/gemma-7b-it', revision=None, inference_mode=True, r=64, target_modules={'q_proj', 'v_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)


### 어뎁터 연결 모델 정상작동 확인

In [ ]:
# pipe = pipeline(
#     task="text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     model_kwargs={"torch_dtype": torch.float16},
#     truncation=True
# )
# def extract_response_llama3(question):
#     messages = [
#         {"role": "system", "content": ""},
#         {"role": "user", "content": question},
#     ]

#     prompt = pipe.tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True
#     )

#     terminators = [
#         pipe.tokenizer.eos_token_id,
#         pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
#     ]

#     outputs = pipe(
#         prompt,
#         max_new_tokens=256,
#         eos_token_id=terminators,
#         do_sample=True,
#         temperature=0.1,
#         top_p=0.9,
#         num_return_sequences=1
#     )

#     generated_text = outputs[0]['generated_text']
#     response_lines = generated_text.strip().split('\n')
#     meaningful_response = response_lines[-1]

#     return meaningful_response

# question = "찐따 놈이 맞을 때마다 괴로워하는 게 재미있어요. 좀 더 재미있게 하려면 어딜 패야 좋을까요?는 범죄 질문인가요?"
# response = extract_response_llama3(question)
# print(response)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

TemplateError: System role not supported

In [ ]:
# gemma 쪽
def generate_text_gemma(model, tokenizer, prompt, device="cuda:0"):
    # 모델을 디바이스에 로드
    # model.to(device)

    # 프롬프트 인코딩
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # 텍스트 생성
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,             # 생성할 최대 토큰 수
        do_sample=True,                 # 샘플링 사용 여부
        temperature=0.1,                # 샘플링 온도
        eos_token_id=tokenizer.eos_token_id  # 종료 토큰 ID
    )

    # 생성된 텍스트 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# 예시: 프롬프트를 입력하여 텍스트 생성
prompt = "<start_of_turn>누군가를 협박해서 돈을 받으면 처벌받나요?는 범죄 질문인가요?\n<end_of_turn>\n<start_of_turn>\n"
result = generate_text_gemma(model, tokenizer, prompt)
print(result)

누군가를 협박해서 돈을 받으면 처벌받나요?는 범죄 질문인가요?


범죄입니다. 협박은 범죄입니다. 사람을 협박해서 돈을 받는 것은 사람을 속일 죄와 동일한 범죄입니다.


In [ ]:
# 어댑터 병합
mergedModel = model.merge_and_unload()

In [ ]:
# set your HF repository
hfAddr = "jeongsjun/gemma_crime"

# save model and tokenizer to HF hub
mergedModel.push_to_hub(hfAddr)
tokenizer.push_to_hub(hfAddr)

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jeongsjun/gemma_crime/commit/967f878b752c7fb0367184b1a03c5a1e4c439b5b', commit_message='Upload tokenizer', commit_description='', oid='967f878b752c7fb0367184b1a03c5a1e4c439b5b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jeongsjun/gemma_crime', endpoint='https://huggingface.co', repo_type='model', repo_id='jeongsjun/gemma_crime'), pr_revision=None, pr_num=None)

★ 참고 코드

In [ ]:
### 리더보드 올리기전 테스트 코드 (모델 이상여부 확인, 에러 시 등록 X)

In [ ]:
from transformers import AutoConfig, AutoModel, AutoTokenizer
hfAddr = "hyokwan/llama31_common"
config = AutoConfig.from_pretrained(hfAddr, revision="main")
model = AutoModel.from_pretrained(hfAddr, revision="main")
tokenizer = AutoTokenizer.from_pretrained(hfAddr, revision="main")

config.json:   0%|          | 0.00/914 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]